In [1]:
#Ignore scikit learn deprication warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


In [2]:
import os
print(os.getcwd())
#os.chdir('../blocking/')
import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re


C:\Users\aparn\Desktop\cs638project\learning


In [6]:
from numpy import genfromtxt
# Awesome sauce.  Now let's fill in missing values rather than dropping them
# reload train
train = genfromtxt('train_reduced.csv',delimiter=',')
test = genfromtxt('unlabeled_reduced.csv',delimiter=',')
#unlabeled = genfromtxt('unlabeled_reduced.csv',delimiter=',')
# get rid of first row (I think this is just column labels)
train = train[1:,:]
test = test[1:,:]




#print(test)
#print(test.shape)
#unlabeled = unlabeled[1:,:]

# # # replace missing featureVals with 0.5 (halfway in between)
# where_are_NaNs = np.isnan(train)
# train[where_are_NaNs] = 0


##Drop all nans:
train = train[~np.isnan(train).any(axis=1)]
test = test[~np.isnan(test).any(axis=1)]
#unlabeled = unlabeled[~np.isnan(unlabeled).any(axis=1)]

y_test = test[:,test.shape[1]-1] # label
X_test = test[:,1:test.shape[1]-1]  # data


y = train[:,train.shape[1]-1] # label
X = train[:,1:train.shape[1]-1]  # data




In [4]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import *
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import *



In [39]:
labeled = em.read_csv_metadata("unlabeled.csv", encoding="ISO-8859-1", key='_id')
labeled_r = em.read_csv_metadata("unlabeled_reduced.csv", encoding="ISO-8859-1", key='_id')
labeled = labeled.dropna(subset = ['_id', 'name_name_jac_qgm_3_qgm_3', 'name_name_jac_dlm_dc0_dlm_dc0',
       'countries_countries_jac_qgm_3_qgm_3',
       'countries_countries_cos_dlm_dc0_dlm_dc0',
       'countries_countries_jac_dlm_dc0_dlm_dc0', 'countries_countries_mel',
       'countries_countries_lev_dist', 'countries_countries_lev_sim',
       'countries_countries_nmw', 'countries_countries_sw',
       'country_count_country_count_exm', 'country_count_country_count_anm',
       'country_count_country_count_lev_dist',
       'country_count_country_count_lev_sim', 'status_match',
       'country_overlap', 'country_count_sim'])
#labeled.head()
#print(test[2])
#labeled_r.columns

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [7]:
# CME's grid search stuff
from sklearn.grid_search import GridSearchCV


# SVC tuning
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=10,
                       scoring='f1')
clf.fit(X,y)

# Get parameters from model
#params = clf.get_params()
#sv = clf.support_vectors
#nv = clf.n_support_
#a  = clf.dual_coef_
#b  = clf._intercept_
#cs = clf.classes_



prediction = clf.predict(X_test)
print(prediction)

print("SVC:" + str(clf.best_score_))
print("SVC:" + str(clf.best_estimator_))


[ 0.  0.  1. ...,  0.  0.  0.]
SVC:0.8240783410138248
SVC:SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [40]:
labeled['labeled'] = prediction
labeled.head()
#len(dropped)

,_id,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,...,countries_countries_nmw,countries_countries_sw,country_count_country_count_exm,country_count_country_count_anm,country_count_country_count_lev_dist,country_count_country_count_lev_sim,status_match,country_overlap,country_count_sim,labeled
0,2103728,6253,265,light red meranti,Shorea,Dipterocarpaceae,Terrestrial,Indonesia (Sumatera); Malaysia (Peninsular Malaysia);,NaN,['\n The species occurs in some forest reserves.\n\n \n '],...,-33.0,2.0,0,0.500000,1.0,0.0,0,0.0,1,0.0
2,7553756,2846,954,"blackfaced black spider monkey, chamek spider monkey, peruvian black spider monkey, peruvian spi...",Ateles,Atelidae,Terrestrial,"Bolivia, Plurinational States of; Brazil (Acre, Amazonas, Mato Grosso, RondÃ?Â´nia); Peru;",\r\r\r\r\n The major threat is subsistence and market hunting for food (with guns). An additi...,"['\n This species is confirmed, or may occur, in numerous protected areas.', <br/>, <br/>, 'B...",...,-87.0,0.0,0,0.000000,1.0,0.0,0,0.0,0,0.0
4,8371955,5800,1057,"black sea bass, giant sea bass",Stereolepis,Polyprionidae,Marine,"<div>Pacific Ã¢?? eastern central</div>; Mexico (Baja California, Sonora); United States (Califo...","\r\r\r\r\n The major threat is from overfishing; however, the species is protected in Califor...",['\n The fish has been protected in California since 1982 and protected in Mexico since 1992 ...,...,-89.0,2.0,0,0.333333,1.0,0.0,1,0.0,0,1.0
5,1780670,7710,224,largefin pupfish,Cyprinodon,Cyprinodontidae,NaN,Mexico;,NaN,NaN,...,-25.0,3.0,0,0.333333,1.0,0.0,0,0.0,0,0.0
7,1840302,4022,232,chamula mountain brook frog,Duellmanohyla,Hylidae,Terrestrial; Freshwater,Mexico;,\r\r\r\r\n The main threat is deforestation and the disturbance of cloud forest remnants (at ...,"['\n The range of this species does not include any protected areas, and the protection and r...",...,-10.0,1.0,0,0.500000,1.0,0.0,1,0.0,1,0.0


NameError: name 'ids' is not defined